In [3]:
import pandas as pd
import numpy as np
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Download necessary NLTK data
nltk.download('stopwords')

# Initialize the stopwords and stemmer
stopword = set(stopwords.words("english"))
stemmer = SnowballStemmer("english")

# Load the dataset
df = pd.read_csv("twitter_data.csv")

# Map the 'class' column to the 'labels' column
df['labels'] = df['class'].map({
    0: "Hate Speech Detected",
    1: "Offensive language detected",
    2: "No hate and Offensive speech"
})

# Keep only the relevant columns
df = df[['tweet', 'labels']]

# Define the text cleaning function
def clean(text):
    text = str(text).lower()
    text = re.sub(r'\[.*?\]', '', text)  # Remove text inside square brackets
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'<.*?>+', '', text)  # Remove HTML tags
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)  # Remove punctuation
    text = re.sub(r'\n', '', text)  # Remove newlines
    text = re.sub(r'\w*\d\w*', '', text)  # Remove words with digits
    text = [word for word in text.split(' ') if word not in stopword]  # Remove stopwords
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]  # Apply stemming
    text = " ".join(text)
    return text

# Apply the clean function to the 'tweet' column
df["tweet"] = df["tweet"].apply(clean)

# Convert text and labels to numpy arrays
x = np.array(df["tweet"])
y = np.array(df["labels"])

# Vectorize the text data using CountVectorizer
cv = CountVectorizer()
x = cv.fit_transform(x)

# Encode labels (since they are string, DecisionTreeClassifier needs them numeric)
le = LabelEncoder()
y = le.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

# Initialize and train the DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Predict and evaluate the model
y_pred = clf.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

# Function to predict new tweets
def predict_tweet(tweet):
    # Clean and vectorize the new tweet
    tweet = clean(tweet)  # Clean the tweet text
    tweet_vect = cv.transform([tweet])  # Transform the cleaned tweet into the same format as the training data
    prediction = clf.predict(tweet_vect)  # Make the prediction

    # Decode the numeric prediction to the corresponding label
    label = le.inverse_transform(prediction)

    return label[0]

# Input: User enters a tweet
user_input = input("Enter a tweet: ")

# Get the prediction for the entered tweet
result = predict_tweet(user_input)
print(f"Prediction: {result}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Accuracy: 0.8756571708032767
Enter a tweet: I Hate you
Prediction: No hate and Offensive speech
